In [46]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy.io import wavfile
import soundfile as sf
import pyaudio
import wave

In [58]:
def setup_graph(title='', x_label='', y_label='', fig_size=None):
    fig = plt.figure()
    if fig_size != None:
        fig.set_size_inches(fig_size[0], fig_size[1])
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

def grabar_senial(ds, path):
    # Definimos parametros
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    CHUNK = 1024
    duracion = ds
    file = path

    # Inicar n
    audio = pyaudio.PyAudio()

    stream = audio.open(
        format=FORMAT,
        channels= CHANNELS,
        rate=RATE,
        frames_per_buffer=CHUNK
    )

    print('Grabando...')
    frames =[]


    for i in range(0, int(RATE/CHUNK*duracion)):
        data = stream.read(CHUNK)
        frames.append(data)
    stream.stop_stream()
    stream.close()
    audio.terminate()


    wavfile = wave.open()

def save_as_wav(file_name, signal, fs):
    # Asegurarse de que la señal sea de tipo entero de 16 bits
    sf.write(file_name, signal, fs)
    print(f"La grabación se guardó en {file_name}.")

###  FSK modulation

In [48]:

def bfsk_modulate(bit_array, space_freq, mark_freq, baud, sample_rate):
    seconds_per_bit = 1 / baud
    samples_per_bit = int(sample_rate * seconds_per_bit)
    t = np.linspace(0, seconds_per_bit, samples_per_bit, endpoint=False)
    signal = np.array([])
    for bit in bit_array:
        if bit == 0:
            space = np.sin(2 * np.pi * space_freq * t)
            signal = np.append(signal, space)
        elif bit == 1:
            mark = np.sin(2 * np.pi * mark_freq * t)
            signal = np.append(signal, mark)
    return signal

### FSK demodulation

In [49]:
import numpy as np

def bfsk_demodulate(signal, space_freq, mark_freq, baud, sample_rate):
    seconds_per_bit = 1 / baud
    samples_per_bit = int(sample_rate * seconds_per_bit)
    bit_array = np.array([])

    for i in range(0, len(signal), samples_per_bit):
        bit_samples = signal[i:i+samples_per_bit]
        correlation_space = np.correlate(bit_samples, np.sin(2 * np.pi * space_freq * np.linspace(0, seconds_per_bit, samples_per_bit, endpoint=False)))
        correlation_mark = np.correlate(bit_samples, np.sin(2 * np.pi * mark_freq * np.linspace(0, seconds_per_bit, samples_per_bit, endpoint=False)))

        if correlation_space > correlation_mark:
            bit_array = np.append(bit_array, 0)
        else:
            bit_array = np.append(bit_array, 1)

    return bit_array.astype(int)

### Modula la cadena de bits y reproduce el audio

In [52]:
sig_010110 = bfsk_modulate([0,1,0,1,1,0], 1000, 2000, 1, 44100)

In [51]:
sd.play(sig_010110, 44100)
sd.wait()

### Guardar .wav
Guarda la senial en un archivo

In [54]:
filename = 'Sonidos/a.wav'

## Demodular

In [42]:
signal_demodulada = bfsk_demodulate(sig_010110, 1000, 2000, 1, 44100)
print(signal_demodulada)

[0 1 0 1 1 0]
